## Mini‑cours pas‑à‑pas : *Market Making avec sélection adverse ("adverse selection" en anglais)*

*(rigoureux mais digeste – les équations **et** l’intuition)*

---

### 0. Que fait un *market maker* ?

* Il **poste des limites** (Bid & Ask) autour d’un prix central (mid‑price) $ mid = \frac{bid+ask}2$.
* Il **gagne le spread** quand un *market order* (MO) touche ses quotes.
* Il porte deux risques :

  1. **Inventaire** : trop long/short ⇒ la PnL devient volatile ;
  2. **Sélection adverse** : un trader mieux informé frappe la quote juste avant un move permanent du prix.

Notre modèle combine ces deux risques dans un cadre « Avellaneda‑Stoikov + sauts ».

---

## 1. Les briques stochastiques

| Élément                        | Formule                                                                                                                            | Lecture « plain language »                                            |
| ------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------- |
| **Mid‑price**                  | \$\displaystyle dS\_t = \sigma,dW\_t + \underbrace{\varepsilon^{+}dM\_t^{+}-\varepsilon^{-}dM\_t^{-}}\_{\text{sauts permanents}}\$ | Bruit diffus + saut \$\pm\varepsilon\$ à chaque MO « informé »        |
| **Arrivées MO**                | \$M\_t^{\pm}\sim\mathcal{P}(\lambda^{\pm}t)\$                                                                                      | En moyenne \$\lambda^{+}\$ achats, \$\lambda^{-}\$ ventes par minute (par exemple mais c'est plus généralement par unité de temps)|
| **Profondeurs**                | Ask \$=S\_t+\delta\_t^{+}\$, Bid \$=S\_t-\delta\_t^{-}\$                                                                           | Décalage par rapport au mid à un instant donné dans la stratégie du market maker                                          |
| **Probabilité d’exécution LO** | \$p^{\pm}\_{hit}=e^{-\kappa^{\pm}\delta^{\pm}}\$                                                                                   | Plus la quote est loin, moins elle est touchée                        |
| **Inventaire**                 | \$Q\_t\$: +1 si on **achète** (Bid touché), −1 si on **vend** (Ask touché)                                                         | Compteur de contrats, actions, bitcoins etc..  détenus à l'instant t de la stratégie                                         |

---

## 2. Objectif du MM : richesse − risque

$$
\max_{\delta^{\pm}}\;\mathbb E\Bigl[\underbrace{X_T+Q_T S_T}_{\text{PnL final}}\! - \alpha Q_T^2
- \phi\!\int_0^T\!Q_u^2 du\Bigr].
$$

* **\$X\_T+Q\_TS\_T\$ est le PnL finale** : le cash + la valeur *(mark‑to‑market)* de l’inventaire.
* **\$\alpha\$ & \$\phi\$** pénalisent respectivement l’inventaire *en fin de journée* et *pendant* la séance.

> *Phrase simple :* l’équation dit : « je veux le plus d’argent possible, mais je déteste finir chargé et je n’aime pas porter du stock trop longtemps ».

---

## 3. Résoudre le contrôle optimal : idée maîtresse

1. **Ansatz additif** : \$H(t,x,S,q)=x+qS+h(t,q)\$ – tout se joue dans \$h\$.
2. Insertion dans l’HJB ⇒ Équation différentielle partielle pour \$h\$.
3. **Profondeurs optimales** :

$$
\boxed{\;\begin{aligned}
\delta^{+,*}(t,q)&=\tfrac1{\kappa^{+}}+\varepsilon^{+}-[h(t,q-1)-h(t,q)],\\[4pt]
\delta^{-,*}(t,q)&=\tfrac1{\kappa^{-}}+\varepsilon^{-}-[h(t,q+1)-h(t,q)].
\end{aligned}\;}
$$

Trois briques indépendantes :

1. \$1/\kappa\$ → **friction** du carnet ;
2. \$\varepsilon\$ → **prime d’assurance info contre les traders informés** ;
3. Différences de \$h\$ → **skew d’inventaire** pour rester toujours le plus non directionnel.

---

## 4. Calcul de $h(t,q)$

### 4.1 Méthode « backward Euler » (discrétisation en temps)

* On discrétise $q \in [\underline q, \bar q]$ (par exemple de –10 à +10).

* On pose une condition terminale :

  $$
  h(T, q) = -\alpha q^2
  $$

* On résout la HJB en remontant le temps, pas à pas, via un **schéma d’Euler implicite (backward Euler)** :

  $$
  h(t - \Delta t, q) = h(t, q) - \Delta t \cdot \text{RHS}(t, q)
  $$

* Cette méthode reste robuste même si le système est non linéaire et asymétrique.
  Complexité numérique : $\mathcal{O}(N \cdot \text{dim})$, où $N$ est le nombre de pas de temps.

---

### 4.2 Méthode matricielle fermée (hypothèse $\kappa^+ = \kappa^- = \kappa$)

On suppose uniquement que le paramètre $\kappa$ est symétrique.
Les intensités $\lambda^{\pm}$ et les impacts $\varepsilon^{\pm}$ peuvent rester **asymétriques**.

#### Étapes de la résolution :

**1.** On pose le changement de variable :

$$
h(t,q) = \frac{1}{\kappa} \log \omega(t,q)
$$

**2.** On empile les $\omega(t,q)$ dans un vecteur :

$$
\omega(t) = \bigl[ \omega(t, \bar q), \omega(t, \bar q - 1), \dots, \omega(t, \underline q) \bigr]^\top
$$

**3.** On construit la matrice tridiagonale $\mathbf{A} \in \mathbb{R}^{d \times d}$ avec $d = \bar q - \underline q + 1$. Ses éléments sont définis par :

$$
\mathbf{A}_{i,q} =
\begin{cases}
q \kappa \left( \varepsilon^+ \lambda^+ - \varepsilon^- \lambda^- \right) - \phi \kappa q^2, & \text{si } i = q \\
\tilde{\lambda}^+, & \text{si } i = q - 1 \\
\tilde{\lambda}^-, & \text{si } i = q + 1 \\
0, & \text{sinon}
\end{cases}
\quad \text{où} \quad
\tilde{\lambda}^\pm = \lambda^\pm \cdot e^{-1 - \kappa \varepsilon^\pm}
$$

**4.** On résout l’équation différentielle matricielle linéaire :

$$
\partial_t \omega(t) + \mathbf{A} \omega(t) = 0
$$

**5.** La solution analytique est :

$$
\boxed{\omega(t) = e^{\mathbf{A}(T - t)} \cdot \mathbf{z}} \quad \text{où} \quad \mathbf{z}_q = e^{- \alpha \kappa q^2}
$$

**6.** Enfin, on retrouve $h(t,q)$ via :

$$
h(t, q) = \frac{1}{\kappa} \log \omega(t, q)
$$

puis on en déduit les profondeurs optimales :

$$
\delta^{+,*}(t,q) = \frac{1}{\kappa} + \varepsilon^+ - \bigl[ h(t, q-1) - h(t, q) \bigr]
$$

$$
\delta^{-,*}(t,q) = \frac{1}{\kappa} + \varepsilon^- - \bigl[ h(t, q+1) - h(t, q) \bigr]
$$

---

## 5. Lecture intuitive des paramètres

| Paramètre       | Interprétation         | Valeurs typiques (futur liquide)    |
| --------------- | ---------------------- | ----------------------------------- |
| \$\sigma\$      | bruit diffus           | 0.10 - 1.0 et plus(dépend de la temporalité)
| \$\lambda\$     | cadence des MOs        | dépend de l'unité de temps par exemple 10/s ou 100/min                          |
| \$\varepsilon\$ | saut instantané permanent après un MO        | dépend de kappa, si $ \varepsilon \times \kappa >= 2 $ marché trop compétitif trop toxic, pnl négatif très probable si $ 1 \leq \varepsilon \times \kappa \leq 2 $ marché compétitif et toxic mais si bonne latence + bon modèle possiblement ok. $ \varepsilon \times \kappa < 1 $ alors pnl possiblement potable si latence correcte, peu de toxicité      |
| \$\kappa\$      | pente du LOB           | dépend de l'unité (dollar, ticksize, bps etc..) |
| \$\alpha\$      | punition fin de séance | \$0.0\$ – \$10^{-1}\$           |
| \$\phi\$        | punition intraday      | \$0.0\$ – \$10^{-1}\$           |

---

## 6. Ce qu’il faut vraiment retenir

1. **Sélection adverse** = risque qu’un trade informé décale le prix contre vous.
2. **\$\varepsilon\$** est le *bouclier* : ajoutez‑le symétriquement au spread pour compenser la perte moyenne.
3. En pratique il faut **estimer dynamiquement** \$\lambda\$, \$\kappa\$, \$\varepsilon\$ en temps réel : statistiques glissantes, filtrage Bayesian, voire *Q‑learning* pour ajuster les quotes *on‑line*.
4. **Latence ultra‑basse** + petit signal d’alpha (ex. *order‑book skew*) deviennent cruciaux ; sinon le modèle seul ne bat pas les concurrents sur des marchés très efficients. Si $ \kappa \times \varepsilon \geq 1.5 $ oubliez sauf si vous voulez un PTSD ou si vous êtes Citadel
5. Le cadre donne un **edge** sur des marchés où :

   * \$\lambda\$ est **élevé** (beaucoup de MOs, donc revenus de spread),

   * \$\kappa\$ est **faible** (carnet creux → on peut charger la marge),

   * \$\varepsilon\$ est **faible** (peu d’info privilégiée).

   > *Traduction :* beaucoup de bruit comportemental, faible efficacité structurelle.

---

**Mantra final** :

$$
\text{demi Spread} = \underbrace{1/\kappa}_{\text{friction}} \; + \; \underbrace{\varepsilon}_{\text{info}} \; + \; \underbrace{\text{skew}(Q)}_{\text{inventaire}}.
$$

Comprenez ces trois briques, calibrez‑les vite et précisément, et ajoutez la vitesse d’exécution : vous tenez la recette du *market making* compétitif.


In [ ]:
!pip install dash jupyter-dash pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.1 MB/s eta 0:00:00


In [ ]:
import dash
from dash import dcc, html, Input, Output, State, callback, Dash
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from scipy.linalg import expm, eig, inv
from scipy.optimize import curve_fit
from scipy.stats import norm, pareto
import pandas as pd
from jupyter_dash import JupyterDash
from dataclasses import dataclass
from typing import Dict, Tuple, List
import logging


app = Dash(__name__)





# FONCTIONS POUR LA SIMULATION DU MARCHE ET BACKTESTING


In [ ]:


def _ffill_bfill_2d(a: np.ndarray) -> np.ndarray:
    """
    Remplit NaN/Inf par propagation :
      1. forward-fill (vers +t)
      2. backward-fill (vers −t) pour les valeurs en tête de colonne
    Retourne la copie corrigée.
    """
    b = a.copy()
    # forward fill
    mask = np.isfinite(b)
    idx  = np.where(mask, np.arange(b.shape[0])[:, None], 0)
    idx  = np.maximum.accumulate(idx, axis=0)
    b    = b[idx, np.arange(b.shape[1])]

    # backward fill (pour ce qui reste non défini en haut de colonne)
    still_bad = ~np.isfinite(b)
    if np.any(still_bad):
        idx = np.where(~still_bad, np.arange(b.shape[0])[:, None], b.shape[0]-1)
        idx = np.minimum.accumulate(idx[::-1, :], axis=0)[::-1, :]
        b[still_bad] = b[idx[still_bad], np.arange(b.shape[1])[still_bad.any(0)]]

    return b
# ───────────────────────────────────────────────────────────────────────────────
def calculate_h_and_depths(params, eps_log: float = 1e-16):
    T, N = params['T'], params['N']
    lam_p, lam_m = params['lambda_p_mod'], params['lambda_m_mod']
    kappa_p, kappa_m = params['kappa_p_mod'], params['kappa_m_mod']
    eps_p_hjb, eps_m_hjb = params['epsilon_p_mod'], params['epsilon_m_mod']
    phi, alpha = params['phi'], params['alpha']
    q_min, q_max = params['q_min'], params['q_max']

    q_grid = np.arange(q_min, q_max + 1)
    d      = q_grid.size
    kappa  = 0.5 * (kappa_p + kappa_m)

    A = np.zeros((d, d))
    lam_tilde_p = lam_p * np.exp(-1.0 - kappa * eps_p_hjb)
    lam_tilde_m = lam_m * np.exp(-1.0 - kappa * eps_m_hjb)

    for i, q in enumerate(q_grid):
        A[i, i]  = q * kappa * (lam_p * eps_p_hjb - lam_m * eps_m_hjb) - phi * kappa * q**2
        if i + 1 < d: A[i, i + 1] = lam_tilde_m
        if i - 1 >= 0: A[i, i - 1] = lam_tilde_p

    z      = np.exp(-alpha * kappa * q_grid**2)
    eigval, V = eig(A)
    V_inv  = inv(V)
    w      = V_inv @ z
    t_grid = np.linspace(0, T, N + 1)
    h_log  = np.empty((N + 1, d))

    for n, t in enumerate(t_grid):
        dt     = T - t
        theta  = eigval.real * dt
        m      = np.max(theta)
        exp_shifted = np.exp(theta - m)
        omega  = V @ (exp_shifted * w)
        log_omega = np.log(np.maximum(omega, eps_log)) + m
        h_log[n]  = log_omega / kappa

    if not np.all(np.isfinite(h_log)):
        h_log = _ffill_bfill_2d(h_log)

    h_qm1  = np.concatenate([h_log[:, :1],  h_log[:, :-1]], axis=1)
    h_qp1  = np.concatenate([h_log[:, 1:],  h_log[:, -1:]], axis=1)
    delta_plus  = (1.0 / kappa) + eps_p_hjb - (h_qm1 - h_log)
    delta_minus = (1.0 / kappa) + eps_m_hjb - (h_qp1 - h_log)
    delta_minus[:, q_grid == q_max] = np.inf
    delta_plus[:,  q_grid == q_min] = np.inf

    return h_log, delta_plus, delta_minus, q_grid, t_grid

def hit_prob(depth, kappa):
    if depth <= 0.0: return 1.0
    p = np.exp(-kappa * depth)
    return 1.0 if p > 1.0 else p

def run_simulation(params, h, delta_plus, delta_minus, q_grid):
    """Simule l'évolution du marché"""
    T, N = params['T'], params['N']
    dt = T / N
    S0, sigma = params['S0'], params['sigma']
    lam_p_sim, lam_m_sim = params['lambda_p_sim'], params['lambda_m_sim']
    kappa_p_sim, kappa_m_sim = params['kappa_p_sim'], params['kappa_m_sim']
    eps_p_sim, eps_m_sim = params['epsilon_p_sim'], params['epsilon_m_sim']
    q_min, q_max = params['q_min'], params['q_max']

    idx_map = {q: i for i, q in enumerate(q_grid)}
    S, Q, X = np.zeros(N + 1), np.zeros(N + 1, dtype=int), np.zeros(N + 1)
    S[0] = S0
    fills_p, fills_m = np.zeros(N + 1, dtype=int), np.zeros(N + 1, dtype=int)
    M_plus, M_minus = np.random.poisson(lam_p_sim * dt, N), np.random.poisson(lam_m_sim * dt, N)

    for n in range(N):
        q_curr = Q[n]
        q_use = q_min if q_curr < q_min else q_max if q_curr > q_max else q_curr
        i = idx_map[q_use]
        d_plus, d_minus = delta_plus[n, i], delta_minus[n, i]

        if M_plus[n] > 0 and hit_prob(d_plus, kappa_p_sim) > 0:
            k_fill = np.random.binomial(M_plus[n], hit_prob(d_plus, kappa_p_sim))
            if k_fill:
                fills_p[n], Q[n], X[n] = k_fill, Q[n] - k_fill, X[n] + k_fill * (S[n] + d_plus)

        if M_minus[n] > 0 and hit_prob(d_minus, kappa_m_sim) > 0:
            k_fill = np.random.binomial(M_minus[n], hit_prob(d_minus, kappa_m_sim))
            if k_fill:
                fills_m[n], Q[n], X[n] = k_fill, Q[n] + k_fill, X[n] - k_fill * (S[n] - d_minus)

        S[n] += M_plus[n] * eps_p_sim - M_minus[n] * eps_m_sim
        S[n] += sigma * np.sqrt(dt) * np.random.randn()
        S[n+1], Q[n+1], X[n+1] = S[n], Q[n], X[n]

    return dict(S=S, Q=Q, X=X, PnL=X + Q * S, fills_p=fills_p, fills_m=fills_m)


# CODE POUR LE DASHBOARD
# !!!!! ATENTION, LE DASHBOARD NE FONCTIONNE QUE DANS GOOGLE CHROME !!!!!!
# SI VOUS ETES SUR CHROME ALORS LANCEZ TOUT LE CODE ET CLIQUEZ SUR LE LIEN CI DESSOUS

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============================================================================
# CONFIGURATION ET CONSTANTES
# ============================================================================

# Palette de couleurs professionnelle
COLORS = {
    'primary': '#63b3ed',
    'secondary': '#f6ad55',
    'success': '#68d391',
    'danger': '#fc8181',
    'info': '#4fd1c5',
    'warning': '#f6e05e',
    'dark': '#1a2332',
    'background': '#0a0f1c',
    'text': '#e0e6ed',
    'muted': '#718096',
    'border': '#2a3f5f'
}

# ============================================================================
# STYLES CSS
# ============================================================================

def get_app_styles():
    """Retourne les styles CSS de l'application"""
    return '''
    <!DOCTYPE html>
    <html>
        <head>
            {%metas%}
            <title>{%title%}</title>
            {%favicon%}
            {%css%}
            <style>
                /* Variables CSS pour une maintenance facile */
                :root {
                    --primary-color: #63b3ed;
                    --secondary-color: #f6ad55;
                    --success-color: #68d391;
                    --danger-color: #fc8181;
                    --info-color: #4fd1c5;
                    --dark-bg: #1a2332;
                    --darker-bg: #0a0f1c;
                    --text-primary: #e0e6ed;
                    --text-secondary: #a0aec0;
                    --text-muted: #718096;
                    --border-color: #2a3f5f;
                    --shadow: 0 5px 15px rgba(0,0,0,0.3);
                    --shadow-hover: 0 10px 30px rgba(0,0,0,0.4);
                }

                * {
                    box-sizing: border-box;
                }

                body {
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                    background-color: var(--darker-bg);
                    color: var(--text-primary);
                    margin: 0;
                    padding: 0;
                    line-height: 1.6;
                }

                .dashboard-container {
                    max-width: 1600px;
                    margin: 0 auto;
                    padding: 20px;
                }

                /* Header amélioré */
                .dashboard-header {
                    background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
                    border-radius: 20px;
                    padding: 40px;
                    text-align: center;
                    margin-bottom: 30px;
                    box-shadow: var(--shadow-hover);
                    position: relative;
                    overflow: hidden;
                }

                .dashboard-header::before {
                    content: '';
                    position: absolute;
                    top: -50%;
                    right: -50%;
                    width: 200%;
                    height: 200%;
                    background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
                    transform: rotate(45deg);
                }

                .dashboard-title {
                    font-size: 2.5rem;
                    font-weight: 700;
                    margin: 0;
                    color: white;
                    position: relative;
                    z-index: 1;
                }

                .dashboard-subtitle {
                    font-size: 1.2rem;
                    color: rgba(255, 255, 255, 0.9);
                    margin-top: 10px;
                    position: relative;
                    z-index: 1;
                }

                /* Sections */
                .section {
                    margin-bottom: 40px;
                }

                .section-header {
                    display: flex;
                    align-items: center;
                    margin-bottom: 25px;
                    padding-bottom: 15px;
                    border-bottom: 2px solid var(--border-color);
                }

                .section-title {
                    font-size: 1.8rem;
                    font-weight: 600;
                    color: var(--primary-color);
                    margin: 0;
                    display: flex;
                    align-items: center;
                }

                .section-icon {
                    width: 30px;
                    height: 30px;
                    margin-right: 15px;
                    background: var(--primary-color);
                    border-radius: 8px;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                }

                /* Cards améliorées */
                .card {
                    background: var(--dark-bg);
                    border-radius: 16px;
                    padding: 25px;
                    box-shadow: var(--shadow);
                    border: 1px solid var(--border-color);
                    transition: all 0.3s ease;
                }

                .card:hover {
                    transform: translateY(-2px);
                    box-shadow: var(--shadow-hover);
                }

                .card-title {
                    font-size: 1.3rem;
                    font-weight: 600;
                    color: var(--text-primary);
                    margin-bottom: 20px;
                }

                /* Grid layouts */
                .parameters-grid {
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(480px, 1fr));
                    gap: 25px;
                    margin-bottom: 30px;
                }

                .metrics-grid {
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 20px;
                    margin-bottom: 30px;
                }

                .graphs-grid {
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(500px, 1fr));
                    gap: 25px;
                }

                /* Sliders améliorés */
                .slider-container {
                    margin-bottom: 25px;
                }

                .slider-label {
                    font-weight: 600;
                    color: var(--text-secondary);
                    margin-bottom: 8px;
                    display: block;
                }

                .param-description {
                    font-size: 0.85rem;
                    color: var(--text-muted);
                    margin-top: 5px;
                    font-style: italic;
                }

                /* Boutons améliorés */
                .btn {
                    border: none;
                    padding: 12px 30px;
                    border-radius: 10px;
                    font-size: 1rem;
                    font-weight: 600;
                    cursor: pointer;
                    transition: all 0.3s ease;
                    text-transform: uppercase;
                    letter-spacing: 0.5px;
                }

                .btn-primary {
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white;
                    box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
                }

                .btn-primary:hover {
                    transform: translateY(-2px);
                    box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4);
                }

                /* Métriques */
                .metric-card {
                    background: linear-gradient(135deg, #2d3748 0%, #1a202c 100%);
                    padding: 25px;
                    border-radius: 12px;
                    text-align: center;
                    border: 1px solid #4a5568;
                    transition: all 0.3s ease;
                }

                .metric-card:hover {
                    transform: scale(1.05);
                    box-shadow: var(--shadow-hover);
                }

                .metric-value {
                    font-size: 2.5rem;
                    font-weight: 700;
                    color: var(--primary-color);
                    margin-bottom: 10px;
                }

                .metric-label {
                    font-size: 0.9rem;
                    color: var(--text-secondary);
                    text-transform: uppercase;
                    letter-spacing: 1px;
                }

                /* Info boxes */
                .info-box {
                    background: rgba(99, 179, 237, 0.1);
                    border: 1px solid var(--primary-color);
                    border-radius: 8px;
                    padding: 15px;
                    margin: 20px 0;
                }

                .info-box-title {
                    font-weight: 600;
                    color: var(--primary-color);
                    margin-bottom: 8px;
                }

                /* Responsive */
                @media (max-width: 768px) {
                    .dashboard-container {
                        padding: 10px;
                    }

                    .parameters-grid,
                    .graphs-grid {
                        grid-template-columns: 1fr;
                    }

                    .dashboard-title {
                        font-size: 1.8rem;
                    }

                    .section-title {
                        font-size: 1.4rem;
                    }
                }

                /* Scrollbar personnalisée */
                ::-webkit-scrollbar {
                    width: 10px;
                    height: 10px;
                }

                ::-webkit-scrollbar-track {
                    background: var(--dark-bg);
                }

                ::-webkit-scrollbar-thumb {
                    background: var(--primary-color);
                    border-radius: 5px;
                }

                ::-webkit-scrollbar-thumb:hover {
                    background: var(--secondary-color);
                }
            </style>
        </head>
        <body>
            {%app_entry%}
            <footer>
                {%config%}
                {%scripts%}
                {%renderer%}
            </footer>
        </body>
    </html>
    '''

def generate_synthetic_market_data(params, noise_level):
    """Génère des données de marché synthétiques"""
    delta_range = np.linspace(0, 0.05, 100)
    n_jumps = 10000

    def get_side_data(lambda_true, kappa_true, epsilon_true):
        prob_true = np.exp(-kappa_true * delta_range)
        prob_observed = np.clip(prob_true + np.random.normal(0, noise_level * 0.1, len(delta_range)), 0, 1)
        lambda_delta_true = lambda_true * np.exp(-kappa_true * delta_range)
        lambda_observed = np.maximum(lambda_delta_true + np.random.normal(0, noise_level * lambda_true * 0.2, len(delta_range)), 0)
        pareto_alpha = 1.5
        if epsilon_true > 0 and pareto_alpha > 1:
            scale = epsilon_true * (pareto_alpha - 1) / pareto_alpha
            jumps_true = pareto.rvs(b=pareto_alpha, scale=scale, size=n_jumps)
        else:
            jumps_true = np.zeros(n_jumps)
        jumps_observed = np.maximum(1e-5, jumps_true + np.random.normal(0, noise_level * epsilon_true * 0.5, n_jumps))
        return prob_true, prob_observed, lambda_delta_true, lambda_observed, jumps_observed

    prob_p_true, prob_p_obs, lam_p_true, lam_p_obs, jumps_p_obs = get_side_data(params['lambda_p_sim'], params['kappa_p_sim'], params['epsilon_p_sim'])
    prob_m_true, prob_m_obs, lam_m_true, lam_m_obs, jumps_m_obs = get_side_data(params['lambda_m_sim'], params['kappa_m_sim'], params['epsilon_m_sim'])

    return {
        'delta_range': delta_range,
        'prob_p_true': prob_p_true, 'prob_p_observed': prob_p_obs,
        'lambda_delta_p_true': lam_p_true, 'lambda_p_observed': lam_p_obs,
        'jumps_p_observed': jumps_p_obs,
        'prob_m_true': prob_m_true, 'prob_m_observed': prob_m_obs,
        'lambda_delta_m_true': lam_m_true, 'lambda_m_observed': lam_m_obs,
        'jumps_m_observed': jumps_m_obs
    }

def calibrate_from_noisy_data(synthetic_data):
    """Calibre le modèle à partir des données"""
    delta_range = synthetic_data['delta_range']

    def fit_side(prob_obs, lambda_obs, jumps_obs):
        mask = prob_obs > 0.01
        kappa_est = 100
        if np.sum(mask) > 10:
            coeffs = np.polyfit(delta_range[mask], np.log(prob_obs[mask]), 1)
            kappa_est = -coeffs[0]

        def lambda_model(delta, lam, kap): return lam * np.exp(-kap * delta)
        try:
            popt, _ = curve_fit(lambda_model, delta_range, lambda_obs, p0=[np.mean(lambda_obs[:5]), kappa_est], bounds=([0, 0], [200, 300]))
            lambda_est_fit, kappa_est_fit = popt
        except:
            lambda_est_fit, kappa_est_fit = np.mean(lambda_obs[:5]), kappa_est
        epsilon_est = np.mean(jumps_obs)
        return kappa_est_fit, lambda_est_fit, epsilon_est

    kp, lp, ep = fit_side(synthetic_data['prob_p_observed'], synthetic_data['lambda_p_observed'], synthetic_data['jumps_p_observed'])
    km, lm, em = fit_side(synthetic_data['prob_m_observed'], synthetic_data['lambda_m_observed'], synthetic_data['jumps_m_observed'])

    return {'kappa_p_est': kp, 'lambda_p_est': lp, 'epsilon_p_est': ep, 'kappa_m_est': km, 'lambda_m_est': lm, 'epsilon_m_est': em}

# ============================================================================
# CRÉATION DE L'APPLICATION
# ============================================================================

app = Dash(__name__)
app.index_string = get_app_styles()

# Paramètres initiaux
def random_model_params():
    return {
        'lambda_plus': 10,
        'lambda_minus': 10,
        'kappa_plus': 110,
        'kappa_minus': 110,
        'epsilon_plus': 0,
        'epsilon_minus': 0,
    }

rnd = random_model_params()

# ============================================================================
# LAYOUT
# ============================================================================

app.layout = html.Div([
    html.Div([
        # Header
        html.Div([
            html.H1("Stratégie de Market Making Optimale", className='dashboard-title'),
            html.P("Simulation avancée avec gestion du risque d'inventaire et de sélection adverse", className='dashboard-subtitle')
        ], className='dashboard-header'),

        # Section 1: Paramètres
        html.Div([
            html.Div([
                html.Div("📊", className='section-icon'),
                html.H2("1. Configuration du Marché et du Modèle", className='section-title')
            ], className='section-header'),

            html.Div(className='parameters-grid', children=[
                # Modèle
                html.Div([
                    html.H3("Modèle (Hypothèse du Market Maker)", className='card-title'),
                    html.P("Paramètres estimés par le market maker", style={'color': COLORS['muted'], 'marginBottom': '20px'}),

                    html.Div([
                        html.Label("λ+ : Intensité des ordres d'achat", className='slider-label'),
                        dcc.Slider(id='lambda-plus-model', min=10, max=100, step=1, value=rnd['lambda_plus'],
                                 marks={str(i): str(i) for i in range(10, 101, 10)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("λ- : Intensité des ordres de vente", className='slider-label'),
                        dcc.Slider(id='lambda-minus-model', min=10, max=100, step=1, value=rnd['lambda_minus'],
                                 marks={str(i): str(i) for i in range(10, 101, 10)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("κ+ : Sensibilité du carnet (achat)", className='slider-label'),
                        dcc.Slider(id='kappa-plus-model', min=10, max=200, step=5, value=rnd['kappa_plus'],
                                 marks={str(i): str(i) for i in range(10, 201, 20)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("κ- : Sensibilité du carnet (vente)", className='slider-label'),
                        dcc.Slider(id='kappa-minus-model', min=10, max=200, step=5, value=rnd['kappa_minus'],
                                 marks={str(i): str(i) for i in range(10, 201, 20)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("ε+ : Impact permanent (achat)", className='slider-label'),
                        dcc.Slider(id='epsilon-plus-model', min=0, max=0.15, step=0.0005, value=rnd['epsilon_plus'],
                                 marks={f"{x:.3f}": f"{x:.3f}" for x in np.linspace(0, 0.15, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("ε- : Impact permanent (vente)", className='slider-label'),
                        dcc.Slider(id='epsilon-minus-model', min=0, max=0.15, step=0.0005, value=rnd['epsilon_minus'],
                                 marks={f"{x:.3f}": f"{x:.3f}" for x in np.linspace(0, 0.15, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),
                ], className='card'),

                # Simulation
                html.Div([
                    html.H3("Simulation (Marché Réel)", className='card-title'),
                    html.P("Vrais paramètres du marché", style={'color': COLORS['muted'], 'marginBottom': '20px'}),

                    html.Div([
                        html.Label("λ+ : Vraie intensité des ordres d'achat", className='slider-label'),
                        dcc.Slider(id='lambda-plus-sim', min=10, max=100, step=1, value=100,
                                 marks={str(i): str(i) for i in range(10, 101, 10)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("λ- : Vraie intensité des ordres de vente", className='slider-label'),
                        dcc.Slider(id='lambda-minus-sim', min=10, max=100, step=1, value=100,
                                 marks={str(i): str(i) for i in range(10, 101, 10)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("κ+ : Vraie sensibilité du carnet (achat)", className='slider-label'),
                        dcc.Slider(id='kappa-plus-sim', min=10, max=200, step=5, value=100,
                                 marks={str(i): str(i) for i in range(10, 201, 20)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("κ- : Vraie sensibilité du carnet (vente)", className='slider-label'),
                        dcc.Slider(id='kappa-minus-sim', min=10, max=200, step=5, value=100,
                                 marks={str(i): str(i) for i in range(10, 201, 20)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("ε+ : Vrai impact permanent (achat)", className='slider-label'),
                        dcc.Slider(id='epsilon-plus-sim', min=0, max=0.15, step=0.0005, value=0.0185,
                                 marks={f"{x:.3f}": f"{x:.3f}" for x in np.linspace(0, 0.15, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("ε- : Vrai impact permanent (vente)", className='slider-label'),
                        dcc.Slider(id='epsilon-minus-sim', min=0, max=0.15, step=0.0005, value=0.0185,
                                 marks={f"{x:.3f}": f"{x:.3f}" for x in np.linspace(0, 0.15, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),
                ], className='card'),
            ]),
        ], className='section'),

        # Section 2: Stratégie
        html.Div([
            html.Div([
                html.Div("⚙️", className='section-icon'),
                html.H2("2. Paramètres de Stratégie et Risque", className='section-title')
            ], className='section-header'),

            html.Div(className='card', children=[
                html.Div(style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(350px, 1fr))', 'gap': '20px'}, children=[
                    html.Div([
                        html.Label("φ : Aversion au risque intraday", className='slider-label'),
                        dcc.Slider(id='phi', min=0, max=0.1, step=0.0005, value=0.1,
                                 marks={f"{x:.4f}": f"{x:.4f}" for x in np.linspace(0, 0.1, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("α : Aversion au risque terminal", className='slider-label'),
                        dcc.Slider(id='alpha', min=0, max=0.1, step=0.0005, value=0.003,
                                 marks={f"{x:.4f}": f"{x:.4f}"  for x in np.linspace(0, 0.1, 6)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("σ : Volatilité du prix", className='slider-label'),
                        dcc.Slider(id='sigma', min=0.1, max=1.0, step=0.01, value=0.5,
                                 marks={f"{x:.2f}": f"{x:.2f}"  for x in np.linspace(0.1, 1.0, 10)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("T : Horizon temporel (minutes)", className='slider-label'),
                        dcc.Slider(id='horizon', min=5, max=30, step=1, value=30,
                                 marks={str(i): str(i) for i in range(5, 31, 5)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("N : Nombre de pas de temps", className='slider-label'),
                        dcc.Slider(id='num-time-steps', min=2000, max=20000, step=1000, value=2000,
                                 marks={str(i): str(i) for i in range(2000, 20001, 2000)},
                                 tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),

                    html.Div([
                        html.Label("Limites d'inventaire", className='slider-label'),
                        dcc.RangeSlider(id='inventory-range', min=-20, max=20, step=1, value=[-4, 4],
                                      marks={str(i): str(i) for i in range(-20, 21, 5)},
                                      tooltip={"placement": "bottom", "always_visible": True})
                    ], className='slider-container'),
                ])
            ])
        ], className='section'),

        # Section 3: Calibration
        html.Div([
            html.Div([
                html.Div("🎯", className='section-icon'),
                html.H2("3. Calibration et Diagnostic", className='section-title')
            ], className='section-header'),

            html.Div(className='card', children=[
                html.H3("Calibration Automatique", className='card-title'),
                html.Div([
                    html.Label("Niveau de bruit dans les données", className='slider-label'),
                    dcc.Slider(id='noise-level', min=0, max=0.5, step=0.05, value=0.15,
                             marks={f"{x:.2f}": f"{x:.2f}" for x in np.linspace(0, 0.5, 6)},
                             tooltip={"placement": "bottom", "always_visible": True})
                ], className='slider-container'),
                html.Div(style={'textAlign': 'center', 'margin': '30px 0'}, children=[
                    html.Button("🔧 Calibrer le Modèle", id='calibrate-model-button', className='btn btn-primary')
                ]),
                html.Div(id='calibration-info', className='info-box')
            ]),

            html.Div(className='graphs-grid', style={'marginTop': '30px'}, children=[
                html.Div(dcc.Graph(id='prob-execution-graph'), className='card'),
                html.Div(dcc.Graph(id='freq-execution-graph'), className='card'),
                html.Div(dcc.Graph(id='price-jump-dist-graph'), className='card'),
            ])
        ], className='section'),

        # Section 4: Résultats
        html.Div([
            html.Div([
                html.Div("📈", className='section-icon'),
                html.H2("4. Résultats de Simulation", className='section-title')
            ], className='section-header'),

            html.Div(className='card', children=[
                html.H3("Métriques de Performance", className='card-title'),
                html.Div(id='metrics-container', className='metrics-grid')
            ]),

            html.Div(dcc.Graph(id='simulation-graph'), className='card', style={'marginTop': '20px'}),

            html.Div(className='graphs-grid', style={'marginTop': '20px'}, children=[
                html.Div(dcc.Graph(id='depth-graph'), className='card'),
                html.Div(dcc.Graph(id='strategy-heatmap'), className='card'),
            ]),

            html.Div([
                html.H3("Visualisations 3D", style={'marginBottom': '20px'}),
                html.Div(className='graphs-grid', children=[
                    html.Div(dcc.Graph(id='3d-ask-depth'), className='card'),
                    html.Div(dcc.Graph(id='3d-bid-depth'), className='card'),
                ])
            ], style={'marginTop': '30px'})
        ], className='section'),

    ], className='dashboard-container')
])

# ============================================================================
# CALLBACKS
# ============================================================================

@app.callback(
    [Output('lambda-plus-model', 'value'), Output('lambda-minus-model', 'value'),
     Output('kappa-plus-model', 'value'), Output('kappa-minus-model', 'value'),
     Output('epsilon-plus-model', 'value'), Output('epsilon-minus-model', 'value'),
     Output('calibration-info', 'children')],
    [Input('calibrate-model-button', 'n_clicks')],
    [State('noise-level', 'value'),
     State('lambda-plus-sim', 'value'), State('lambda-minus-sim', 'value'),
     State('kappa-plus-sim', 'value'), State('kappa-minus-sim', 'value'),
     State('epsilon-plus-sim', 'value'), State('epsilon-minus-sim', 'value')]
)
def suggest_calibration(n_clicks, noise_level, lp_sim, lm_sim, kp_sim, km_sim, ep_sim, em_sim):
    if n_clicks is None:
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, "Cliquez sur 'Calibrer le Modèle' pour estimer les paramètres"

    params_sim = {'lambda_p_sim': lp_sim, 'lambda_m_sim': lm_sim, 'kappa_p_sim': kp_sim, 'kappa_m_sim': km_sim, 'epsilon_p_sim': ep_sim, 'epsilon_m_sim': em_sim}
    synthetic_data = generate_synthetic_market_data(params_sim, noise_level)
    calib = calibrate_from_noisy_data(synthetic_data)

    info_text = f"""**Résultats du Calibrage (Bruit: {noise_level:.2f})**
Achat: λ={calib['lambda_p_est']:.1f} (vrai: {lp_sim}), κ={calib['kappa_p_est']:.1f} (vrai: {kp_sim}), ε={calib['epsilon_p_est']:.4f} (vrai: {ep_sim:.4f})
Vente: λ={calib['lambda_m_est']:.1f} (vrai: {lm_sim}), κ={calib['kappa_m_est']:.1f} (vrai: {km_sim}), ε={calib['epsilon_m_est']:.4f} (vrai: {em_sim:.4f})"""

    return calib['lambda_p_est'], calib['lambda_m_est'], calib['kappa_p_est'], calib['kappa_m_est'], calib['epsilon_p_est'], calib['epsilon_m_est'], info_text

@app.callback(
    [Output('simulation-graph', 'figure'), Output('depth-graph', 'figure'),
     Output('prob-execution-graph', 'figure'), Output('freq-execution-graph', 'figure'),
     Output('price-jump-dist-graph', 'figure'), Output('strategy-heatmap', 'figure'),
     Output('3d-ask-depth', 'figure'), Output('3d-bid-depth', 'figure'),
     Output('metrics-container', 'children')],
    [Input('lambda-plus-model', 'value'), Input('lambda-minus-model', 'value'),
     Input('kappa-plus-model', 'value'), Input('kappa-minus-model', 'value'),
     Input('epsilon-plus-model', 'value'), Input('epsilon-minus-model', 'value'),
     Input('lambda-plus-sim', 'value'), Input('lambda-minus-sim', 'value'),
     Input('kappa-plus-sim', 'value'), Input('kappa-minus-sim', 'value'),
     Input('epsilon-plus-sim', 'value'), Input('epsilon-minus-sim', 'value'),
     Input('phi', 'value'), Input('alpha', 'value'), Input('sigma', 'value'),
     Input('horizon', 'value'), Input('num-time-steps', 'value'),
     Input('inventory-range', 'value'), Input('noise-level', 'value')]
)
def update_all_graphs(lp_mod, lm_mod, kp_mod, km_mod, ep_mod, em_mod,
                      lp_sim, lm_sim, kp_sim, km_sim, ep_sim, em_sim,
                      phi, alpha, sigma, T, N, q_range, noise_level):

    q_min, q_max = q_range
    params = {
        'T': T, 'N': N, 'phi': phi, 'alpha': alpha, 'sigma': sigma,
        'S0': 100.0, 'q_min': q_min, 'q_max': q_max,
        'lambda_p_mod': lp_mod, 'lambda_m_mod': lm_mod, 'kappa_p_mod': kp_mod, 'kappa_m_mod': km_mod,
        'lambda_p_sim': lp_sim, 'lambda_m_sim': lm_sim, 'kappa_p_sim': kp_sim, 'kappa_m_sim': km_sim,
        'epsilon_p_sim': ep_sim, 'epsilon_m_sim': em_sim, 'epsilon_p_mod': ep_mod, 'epsilon_m_mod': em_mod,
    }

    h, delta_plus, delta_minus, q, t_grid = calculate_h_and_depths(params)
    sim_results = run_simulation(params, h, delta_plus, delta_minus, q)
    synthetic_data = generate_synthetic_market_data(params, noise_level)

    # Fig 1: Simulation
    fig1 = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                         subplot_titles=('Profit & Loss (PnL)', 'Inventaire (Q)', 'Prix Mid-Market (S)', 'Exécutions Cumulées'),
                         row_heights=[0.3, 0.25, 0.25, 0.2])
    fig1.add_trace(go.Scatter(x=t_grid, y=sim_results['PnL'], name='PnL', line=dict(color=COLORS['primary'], width=2)), row=1, col=1)
    fig1.add_trace(go.Scatter(x=t_grid, y=sim_results['Q'], name='Inventaire', line=dict(color=COLORS['secondary'], width=2)), row=2, col=1)
    fig1.add_hline(y=q_min, line_dash="dash", line_color="red", row=2, col=1)
    fig1.add_hline(y=q_max, line_dash="dash", line_color="red", row=2, col=1)
    fig1.add_trace(go.Scatter(x=t_grid, y=sim_results['S'], name='Prix', line=dict(color=COLORS['success'], width=1.5)), row=3, col=1)
    fig1.add_trace(go.Scatter(x=t_grid, y=np.cumsum(sim_results['fills_p']), name='Ventes (Ask)', line=dict(color=COLORS['danger'], width=2)), row=4, col=1)
    fig1.add_trace(go.Scatter(x=t_grid, y=np.cumsum(sim_results['fills_m']), name='Achats (Bid)', line=dict(color=COLORS['info'], width=2)), row=4, col=1)
    fig1.update_layout(template='plotly_dark', height=800, title_text='Trajectoires de la Simulation',
                      paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)', xaxis4_title='Temps (minutes)')

    # Fig 2: Depths
    fig2 = go.Figure()
    q_plot = [q_min, q_min//2, 0, q_max//2, q_max]
    colors = px.colors.qualitative.Set3
    for i, qq in enumerate(q_plot):
        if qq in q:
            idx_q = np.where(q == qq)[0][0]
            ask_d, bid_d = delta_plus[:, idx_q].copy(), delta_minus[:, idx_q].copy()
            ask_d[np.isinf(ask_d)], bid_d[np.isinf(bid_d)] = np.nan, np.nan
            fig2.add_trace(go.Scatter(x=t_grid, y=ask_d, name=f'Ask (q={qq})', line=dict(color=colors[i % len(colors)], dash='solid')))
            fig2.add_trace(go.Scatter(x=t_grid, y=bid_d, name=f'Bid (q={qq})', line=dict(color=colors[i % len(colors)], dash='dash')))
    fig2.update_layout(title='Profondeurs Optimales de Cotation', xaxis_title='Temps (minutes)', yaxis_title='Profondeur ($)',
                      template='plotly_dark', height=500, paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)')

    # Fig 3: P(fill)
    fig3 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Achat (+)", "Vente (-)"))
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['prob_p_observed'], mode='markers', name='Observé', marker=dict(color=COLORS['danger'], opacity=0.6)), row=1, col=1)
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['prob_p_true'], name='Vrai', line=dict(color=COLORS['danger'], width=3)), row=1, col=1)
    prob_p_fit = np.exp(-kp_mod * synthetic_data['delta_range'])
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=prob_p_fit, name='Modèle', line=dict(color='white', width=2, dash='dash')), row=1, col=1)
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['prob_m_observed'], mode='markers', marker=dict(color=COLORS['info'], opacity=0.6), showlegend=False), row=2, col=1)
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['prob_m_true'], line=dict(color=COLORS['info'], width=3), showlegend=False), row=2, col=1)
    prob_m_fit = np.exp(-km_mod * synthetic_data['delta_range'])
    fig3.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=prob_m_fit, line=dict(color='white', width=2, dash='dash'), showlegend=False), row=2, col=1)
    fig3.update_layout(title_text="Probabilité d'Exécution P(fill) = exp(-κδ)", template='plotly_dark', height=500,
                      paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)', xaxis2_title='Profondeur δ ($)')

    # Fig 4: Freq(fill)
    fig4 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Achat (+)", "Vente (-)"))
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['lambda_p_observed'], mode='markers', name='Observé', marker=dict(color=COLORS['danger'], opacity=0.6)), row=1, col=1)
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['lambda_delta_p_true'], name='Vrai', line=dict(color=COLORS['danger'], width=3)), row=1, col=1)
    lambda_p_fit = lp_mod * np.exp(-kp_mod * synthetic_data['delta_range'])
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=lambda_p_fit, name='Modèle', line=dict(color='white', width=2, dash='dash')), row=1, col=1)
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['lambda_m_observed'], mode='markers', marker=dict(color=COLORS['info'], opacity=0.6), showlegend=False), row=2, col=1)
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=synthetic_data['lambda_delta_m_true'], line=dict(color=COLORS['info'], width=3), showlegend=False), row=2, col=1)
    lambda_m_fit = lm_mod * np.exp(-km_mod * synthetic_data['delta_range'])
    fig4.add_trace(go.Scatter(x=synthetic_data['delta_range'], y=lambda_m_fit, line=dict(color='white', width=2, dash='dash'), showlegend=False), row=2, col=1)
    fig4.update_layout(title_text="Intensité des Ordres λ(δ) = λ exp(-κδ)", template='plotly_dark', height=500,
                      paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)', xaxis2_title='Profondeur δ ($)')

    # Fig 5: Jumps
    fig5 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Achat (+)", "Vente (-)"))
    fig5.add_trace(go.Histogram(x=synthetic_data['jumps_p_observed'], name='Observé', histnorm='probability density', marker=dict(color=COLORS['danger'], opacity=0.6)), row=1, col=1)
    fig5.add_vline(x=ep_sim, line_dash="dash", line_color=COLORS['danger'], annotation_text="ε Vrai", row=1, col=1)
    fig5.add_vline(x=ep_mod, line_dash="dot", line_color="white", annotation_text="ε Modèle", row=1, col=1)
    fig5.add_trace(go.Histogram(x=synthetic_data['jumps_m_observed'], histnorm='probability density', marker=dict(color=COLORS['info'], opacity=0.6), showlegend=False), row=2, col=1)
    fig5.add_vline(x=em_sim, line_dash="dash", line_color=COLORS['info'], annotation_text="ε Vrai", row=2, col=1)
    fig5.add_vline(x=em_mod, line_dash="dot", line_color="white", annotation_text="ε Modèle", row=2, col=1)
    fig5.update_layout(title_text="Impact Permanent ε = E[Saut de Prix]", template='plotly_dark', height=500,
                      paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)', xaxis2_title="Taille de l'impact ($)")

    # Fig 6: Heatmap
    fig6 = go.Figure()
    t_sample = np.linspace(0, T, 50) if N > 0 else np.array([0])
    h_sample = np.array([h[int(t * (N-1) / T)] for t in t_sample])
    fig6.add_trace(go.Heatmap(z=h_sample.T, x=t_sample, y=q, colorscale='Viridis', colorbar=dict(title='h(t,q)')))
    fig6.update_layout(title="Fonction de Valeur h(t,q)", xaxis_title='Temps (minutes)', yaxis_title='Inventaire (q)',
                      template='plotly_dark', height=500, paper_bgcolor=COLORS['dark'], plot_bgcolor='rgba(0,0,0,0.3)')

    # Fig 7 & 8: 3D
    t_3d = np.linspace(0, T, 50) if N > 0 else np.array([0])
    ask_surface = np.array([delta_plus[int(t * (N-1) / T)] for t in t_3d])
    bid_surface = np.array([delta_minus[int(t * (N-1) / T)] for t in t_3d])
    ask_surface[np.isinf(ask_surface)], bid_surface[np.isinf(bid_surface)] = np.nan, np.nan

    fig7 = go.Figure(data=[go.Surface(x=q, y=t_3d, z=ask_surface.T, colorscale='Reds', colorbar=dict(title='Profondeur Ask'))])
    fig7.update_layout(title='Surface de Profondeur Ask δ+', scene=dict(xaxis_title='Inventaire (q)', yaxis_title='Temps (t)', zaxis_title='Profondeur ($)'),
                      template='plotly_dark', height=600, paper_bgcolor=COLORS['dark'])

    fig8 = go.Figure(data=[go.Surface(x=q, y=t_3d, z=bid_surface.T, colorscale='Blues', colorbar=dict(title='Profondeur Bid'))])
    fig8.update_layout(title='Surface de Profondeur Bid δ-', scene=dict(xaxis_title='Inventaire (q)', yaxis_title='Temps (t)', zaxis_title='Profondeur ($)'),
                      template='plotly_dark', height=600, paper_bgcolor=COLORS['dark'])

    # Métriques
    final_pnl = sim_results['PnL'][-1]
    total_trades = np.sum(sim_results['fills_p']) + np.sum(sim_results['fills_m'])
    max_inventory = np.max(np.abs(sim_results['Q']))
    pnl_diff = np.diff(sim_results['PnL'])
    sharpe = np.mean(pnl_diff) / np.std(pnl_diff) * np.sqrt(N/T) if np.std(pnl_diff) > 0 else 0

    metrics_layout = [
        html.Div([html.Div(f"{final_pnl:.2f} $", className='metric-value'), html.Div("PnL Final", className='metric-label')], className='metric-card'),
        html.Div([html.Div(f"{total_trades}", className='metric-value'), html.Div("Trades Exécutés", className='metric-label')], className='metric-card'),
        html.Div([html.Div(f"{max_inventory}", className='metric-value'), html.Div("Inventaire Max", className='metric-label')], className='metric-card'),
        html.Div([html.Div(f"{sharpe:.3f}", className='metric-value'), html.Div("Ratio de Sharpe", className='metric-label')], className='metric-card')
    ]

    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, metrics_layout

if __name__ == '__main__':
    app.run(jupyter_mode='external', debug=True, height=1500, width="100%")

Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>